In [3]:
import gradio as gr
import os


# Demo

In [24]:
def start_game(main_loop, share=False):

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder='What do you do next?', container=False, scale=7),
        title='AI RPG',
        theme='soft',
        examples=['Look Around', 'Continue the story'],
        cache_examples=False
    )

    demo.launch(share=share, server_name='0.0.0.0')

def test_main_loop(message, history):
    return 'Entered Action: '+message

start_game(test_main_loop)

/Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


# Generating an Initial Start

In [16]:
from helper import load_world, save_world, get_together_api_key, load_env
from together import Together

client = Together(api_key=get_together_api_key())

world = load_world('./data/Westeros.json')
kingdom = world['kingdoms']['Emberhaven']
town = kingdom['towns']["Dragon's Watch"]
character = town['npcs']['Eira Shadowglow']

In [17]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""

world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

In [18]:
model_output = client.chat.completions.create(
    model='meta-llama/Llama-3-70b-chat-hf',
    temperature=1.0,
    messages=[
        {'role':'system', 'content':system_prompt},
        {'role':'user', 'content':world_info+'\n Your Start:'}
    ]
)

In [19]:
start = model_output.choices[0].message.content
print(start)

You stand atop the northern battlements of Dragon's Watch, the wind whipping your short silver hair into a frenzy as you gaze out upon the Wyrmbreaker Mountains. The sun is setting, casting a warm orange glow over the rugged landscape, and you can't help but feel a sense of unease settling in the pit of your stomach. As a Guildmaster of the esteemed Dragon's Watch, you've dedicated your life to understanding and protecting the ancient beasts that roam Drakonaria. But with the recent rumors of dark magic and whispers of conspiracy, you can't shake the feeling that something is amiss.


In [20]:
# Save the start
world['start'] = start
save_world(world, './data/Westeros.json')

# Creating the Main Action Loop

In [21]:
def run_action(message, history, game_state):
    if message == 'start_game':
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""

    world_info = f"""
    World: {game_state['world']}
    Kingdom: {game_state['kingdom']}
    Town: {game_state['town']}
    Your Character: {game_state['character']}"""

    messages = [
        {'role':'system', 'content':system_prompt},
        {'role':'user', 'content':world_info}
    ]

    for action in history:
        messages.append({'role':'assistant', 'content':action[0]})
        messages.append({'role':'user', 'content':action[1]})
    
    messages.append({'role':'user', 'content':message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [22]:
game_state = {
    "world": world['description'],
    "kingdom": kingdom['description'],
    "town": town['description'],
    "character": character['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [25]:
start_game(main_loop, True)

/Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7861): address already in use


* Running on local URL:  http://0.0.0.0:7862

Could not create share link. Missing file: /Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio
